In [1]:
import os
import json
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from glob import glob
from tqdm import tqdm
import pyproj
import matplotlib

import matplotlib.image as mpimg

matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

base_dir = os.path.join(os.path.join(os.path.dirname(os.path.dirname((os.getcwd()))), "data"), "vehicle-to-pedestrian_interaction")
timestamp_dir = os.path.join(os.path.join(base_dir, "raw_virtual"), "Trial_1_timestamp")
real_vehicle_file = os.path.join(os.path.join(base_dir, "raw_virtual"), "Trial_1_Tucsan_Received.txt")

withi = "virtual_pedestrian_scenario_with_intelligence"
withouti = "virtual_pedestrian_scenario_without_intelligence"

veh_length = 4.67

class UTMConverter:
    def __init__(self, zone_number, northern_hemisphere=True):
        utm_crs = pyproj.CRS(f"EPSG:326{zone_number}" if northern_hemisphere else f"EPSG:327{zone_number}")
        wgs84_crs = pyproj.CRS("EPSG:4326")
        self.transformer = pyproj.Transformer.from_crs(utm_crs, wgs84_crs, always_xy=True)

    def convert(self, easting, northing):
        lon, lat = self.transformer.transform(easting, northing)
        return lat, lon

converter = UTMConverter(zone_number=52)

veh_length = 4.67

def recover_position(pos_str, heading):
    PosX, PosZ, PosY = map(float, pos_str.split(", "))
    playerRotY = ((np.pi / 180) * (-float(heading)))
    displace = ((0.5 * veh_length) - 0.785)
    caliX = displace * np.sin(np.radians(playerRotY))
    caliZ = displace * np.cos(np.radians(playerRotY))
    Xdisp = PosX - caliX
    Zdisp = PosZ - caliZ
    realPosZ = (-Xdisp) + 4028673.294 + 0.3165
    realPosX = Zdisp + 356422.983 + 1.63995
    realPosY = PosY - 0.3725 + 53.5840364253 - 0.033078
    return realPosX, realPosZ, realPosY

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

real_data = []
with open(real_vehicle_file, "r") as f:
    for line in f:
        try:
            record = json.loads(line.strip())
            lonr, latr, alt = map(float, record["position"].split(", "))
            lat, lon = converter.convert(lonr, latr)
            real_data.append({
                "timestamp": float(record["t_veh_sent"]),
                "z": latr,
                "x": lonr,
                "heading": float(record["heading"])
            })
        except:
            continue
real_df_all = pd.DataFrame(real_data).dropna()

df_real_all = {cond: [] for cond in [withi, withouti]}

df_withi = []
#pd.DataFrame([])
df_withoi = []
#pd.DataFrame([])

for condition in [withi, withouti]:
    condition_path = os.path.join(timestamp_dir, condition)
    for json_file in glob(os.path.join(condition_path, "*.json")):
        with open(json_file, "r") as f:
            data = json.load(f)
        entries = data.get("DataEntries", [])
        if not entries:
            continue
        test_start = float(entries[0]["Timestamp"])
        test_end = float(entries[-1]["Timestamp"])

        real_df = real_df_all[(real_df_all["timestamp"] >= test_start) & (real_df_all["timestamp"] <= test_end)].copy()
        
        real_df = real_df.sort_values("timestamp")

        real_df["elapsed_time"] = real_df["timestamp"] - test_start
        
        if(condition == "virtual_pedestrian_scenario_with_intelligence"):
            df_withi.append(real_df)
        elif (condition == "virtual_pedestrian_scenario_without_intelligence"):
            df_withoi.append(real_df)

print(df_withi)
print(df_withoi)

[          timestamp         z          x    heading  elapsed_time
45144  1.744960e+09 -80.43040 -55.839720  90.968666      0.045353
45145  1.744960e+09 -80.43050 -55.839720  90.972214      0.103848
45146  1.744960e+09 -80.43064 -55.839730  90.977417      0.153854
45147  1.744960e+09 -80.43071 -55.589730  90.980103      0.203947
45148  1.744960e+09 -80.43082 -55.589730  90.983887      0.253978
...             ...       ...        ...        ...           ...
45369  1.744960e+09 -48.41714  -2.602519   1.367136     27.195709
45370  1.744960e+09 -48.04216  -2.601809   1.393396     27.305935
45371  1.744960e+09 -47.35470  -2.600138   1.455185     27.475705
45372  1.744960e+09 -47.22972  -2.599423   1.481617     27.525707
45373  1.744960e+09 -47.01099  -2.598638   1.510630     27.576225

[230 rows x 5 columns],           timestamp         z          x    heading  elapsed_time
54398  1.744961e+09 -80.53331 -55.340930  92.463104      0.060222
54399  1.744961e+09 -80.56581 -55.090980  92.50921

In [2]:
def _angle_wrap_deg(a):
    a = (a + 180.0) % 360.0 - 180.0
    return -180.0 if a == 180.0 else a

def _linear_drift_safe(time_s, resid, nan_policy="keep"):
    t = np.asarray(time_s, float)
    y = np.asarray(resid, float)
    m = np.isfinite(t) & np.isfinite(y)
    t, y = t[m], y[m]
    if t.size < 3:
        return 0.0 if nan_policy == "zero" else np.nan
    x = t - t[0]
    span = x[-1] - x[0]
    if span <= 1e-9:
        return 0.0 if nan_policy == "zero" else np.nan
    slope, _ = np.polyfit(x, y, 1)
    return float(slope)

def _xcorr_lag_seconds_safe(a, b, time_s, nan_policy="keep"):
    a = np.asarray(a, float)
    b = np.asarray(b, float)
    t = np.asarray(time_s, float)
    m = np.isfinite(a) & np.isfinite(b) & np.isfinite(t)
    a, b, t = a[m], b[m], t[m]
    if a.size < 3:
        return 0.0 if nan_policy == "zero" else np.nan
    dt = np.median(np.diff(t))
    if not np.isfinite(dt) or dt <= 0:
        return 0.0 if nan_policy == "zero" else np.nan
    a = a - a.mean()
    b = b - b.mean()
    xcorr = np.correlate(a, b, mode='full')
    lags = np.arange(-len(a)+1, len(a))
    lag_steps = lags[int(np.argmax(xcorr))]
    return float(lag_steps * dt)

def _sanitize_time(df, time_col_preference=("timestamp","elapsed_time")):
    time_col = next((c for c in time_col_preference if c in df.columns), None)
    if time_col is None:
        raise ValueError("No suitable time column found.")

    d = df.copy()
    d = d[np.isfinite(d[time_col].to_numpy(dtype=float))]
    if d.duplicated(time_col).any():
        num = d.select_dtypes(include="number").columns.tolist()
        d = d.groupby(time_col, as_index=False)[num].mean()
    d = d.sort_values(time_col).reset_index(drop=True)
    return d, time_col

def smooth_and_metrics_robust(df,
                              heading_is_deg=False,
                              nan_policy="keep", 
                              buf_size=5,
                              proc_noise=0.02,
                              meas_noise=0.25,
                              lateral_suppress_speed=0.1,
                              lateral_factor=0.9,
                              heading_alpha=0.8):
    if df.empty:
        raise ValueError("Empty dataframe")

    d, tcol = _sanitize_time(df)

    t = d[tcol].to_numpy(dtype=float)
    x = d["x"].to_numpy(dtype=float)
    z = d["z"].to_numpy(dtype=float)


    if heading_is_deg:
        yaw_deg_raw = -d["heading"].to_numpy(dtype=float)  
    else:
        yaw_deg_raw = -np.degrees(d["heading"].to_numpy(dtype=float))

    x_buf, z_buf = [], []
    pos = np.array([x[0], z[0]], dtype=float)
    vel = np.array([0.0, 0.0], dtype=float)
    unc = 1.0
    Q, R = float(proc_noise), float(meas_noise)

    x_smt = np.zeros_like(x)
    z_smt = np.zeros_like(z)
    heading_smt_deg = np.zeros_like(yaw_deg_raw)
    heading_smt_deg[0] = yaw_deg_raw[0]

    t_prev = t[0]
    for i in range(len(d)):
        dt = max(0.0, float(t[i] - t_prev)) if i > 0 else 0.0
        t_prev = t[i]

        x_buf.append(x[i]); z_buf.append(z[i])
        if len(x_buf) > buf_size:
            x_buf.pop(0); z_buf.pop(0)
        meas = np.array([np.mean(x_buf), np.mean(z_buf)], float)

        pred_pos = pos + vel * dt
        pred_unc = unc + Q
        resid = meas - pred_pos

        speed = np.linalg.norm(vel)
        if speed < lateral_suppress_speed:
            if i > 0 and (abs(x[i]-x[i-1]) + abs(z[i]-z[i-1])) > 1e-9:
                fwd = np.array([x[i]-x[i-1], z[i]-z[i-1]], float)
            else:
                yaw_rad = math.radians(yaw_deg_raw[i])
                fwd = np.array([math.sin(yaw_rad), math.cos(yaw_rad)], float)
            n = fwd / (np.linalg.norm(fwd) + 1e-9)
            lat = resid - np.dot(resid, n) * n
            resid = resid - lateral_factor * lat

        K = max(0.0, min(1.0, pred_unc / (pred_unc + R)))
        pos = pred_pos + K * resid

        meas_vel = resid / max(dt, 1e-3)
        vel = (1.0 - 0.2) * vel + 0.2 * meas_vel

        unc = (1.0 - K) * pred_unc
        pos = (1.0 - 0.8) * pos + 0.8 * (pred_pos + K * resid)

        x_smt[i], z_smt[i] = pos[0], pos[1]

        h_prev = heading_smt_deg[i-1] if i > 0 else yaw_deg_raw[i]
        diff = _angle_wrap_deg(yaw_deg_raw[i] - h_prev)
        heading_smt_deg[i] = _angle_wrap_deg(h_prev + heading_alpha * diff)

    dx = x_smt - x
    dz = z_smt - z
    pos_err_planar = np.sqrt(dx*dx + dz*dz)
    heading_err_deg = np.array([_angle_wrap_deg(heading_smt_deg[i] - yaw_deg_raw[i]) for i in range(len(d))])

    pos_err_mean = float(np.nanmean(pos_err_planar))
    pos_err_var  = float(np.nanvar(pos_err_planar, ddof=1)) if len(pos_err_planar) > 1 else 0.0
    heading_err_mean = float(np.nanmean(np.abs(heading_err_deg)))
    heading_err_var  = float(np.nanvar(heading_err_deg, ddof=1)) if len(heading_err_deg) > 1 else 0.0

    drift_dx_mps = _linear_drift_safe(t, dx, nan_policy=nan_policy)
    drift_dz_mps = _linear_drift_safe(t, dz, nan_policy=nan_policy)
    drift_pos_err_mps = _linear_drift_safe(t, pos_err_planar, nan_policy=nan_policy)
    drift_heading_degps = _linear_drift_safe(t, heading_err_deg, nan_policy=nan_policy)

    sync_lag_s_x = _xcorr_lag_seconds_safe(x, x_smt, t, nan_policy=nan_policy)
    sync_lag_s_z = _xcorr_lag_seconds_safe(z, z_smt, t, nan_policy=nan_policy)

    df_out = d.copy()
    df_out["x_smt"] = x_smt
    df_out["z_smt"] = z_smt
    df_out["heading_smt_deg"] = heading_smt_deg
    df_out["pos_err_planar"] = pos_err_planar
    df_out["heading_err_deg"] = heading_err_deg

    metrics = {
        "pos_err_planar_mean": pos_err_mean,
        "pos_err_planar_var":  pos_err_var,
        "heading_err_deg_mean": heading_err_mean,
        "heading_err_deg_var":  heading_err_var,
        "drift_dx_mps": drift_dx_mps,
        "drift_dz_mps": drift_dz_mps,
        "drift_pos_err_mps": drift_pos_err_mps,
        "drift_heading_degps": drift_heading_degps
    }
    return df_out, metrics

In [3]:
smt_dfwithi, metrics_dfwithi = [], []
for df in df_withi:
    df_smt, m = smooth_and_metrics_robust(df, heading_is_deg=True, nan_policy="zero")
    smt_dfwithi.append(df_smt); metrics_dfwithi.append(m)

smt_dfwithoi, metrics_df_withoi = [], []
for df in df_withoi:
    df_smt, m = smooth_and_metrics_robust(df, heading_is_deg=True, nan_policy="zero")
    smt_dfwithoi.append(df_smt); metrics_df_withoi.append(m)

In [4]:
def summarize_metrics_dicts(metrics_list, condition_name: str) -> pd.DataFrame:
    if not metrics_list:
        return pd.DataFrame(columns=["metric","condition","mean","var","median","p95","min","max","n_tests"])

    df = pd.DataFrame(metrics_list)
    num = df.apply(pd.to_numeric, errors="coerce")

    means   = num.mean(axis=0, skipna=True)
    medians = num.median(axis=0, skipna=True)
    p95s    = num.quantile(0.95, axis=0, interpolation="linear")
    mins    = num.min(axis=0, skipna=True)
    maxs    = num.max(axis=0, skipna=True)
    counts  = num.count(axis=0)
    vars_   = num.var(axis=0, ddof=1, skipna=True) 

    out = pd.DataFrame({
        "metric":  num.columns,
        "condition": condition_name,
        "mean":    means.values,
        "var":     vars_.values,
        "median":  medians.values,
        "p95":     p95s.values,
        "min":     mins.values,
        "max":     maxs.values,
        "n_tests": counts.values,
    })
    return out

def compare_conditions(with_summary: pd.DataFrame,
                       without_summary: pd.DataFrame,
                       metric_names=None,
                       drop_all_nan=True) -> pd.DataFrame:
    w  = with_summary.set_index("metric")[["mean"]]
    wo = without_summary.set_index("metric")[["mean"]]

    if metric_names is None:
        idx = w.index.union(wo.index)
    else:
        idx = pd.Index(metric_names)

    with_mean    = w.reindex(idx)["mean"]
    without_mean = wo.reindex(idx)["mean"]

    comp = pd.DataFrame({
        "metric": idx,
        "with_mean": with_mean.values,
        "without_mean": without_mean.values
    })

    comp["delta"]   = comp["with_mean"] - comp["without_mean"]
    comp["delta_%"] = 100.0 * comp["delta"] / without_mean.replace(0.0, np.nan).values

    if drop_all_nan:
        comp = comp.dropna(how="all", subset=["with_mean","without_mean"])

    return comp[["metric","with_mean","without_mean","delta","delta_%"]]

with_i_summary     = summarize_metrics_dicts(metrics_dfwithi,   "with_intelligence")
without_i_summary  = summarize_metrics_dicts(metrics_df_withoi, "without_intelligence")

key_metrics = [
    "pos_err_planar_mean", "pos_err_planar_var",
    "heading_err_deg_mean", "heading_err_deg_var",
    "drift_dx_mps", "drift_dz_mps", "drift_pos_err_mps", "drift_heading_degps"
]

comparison = compare_conditions(with_i_summary, without_i_summary, metric_names=key_metrics)

print("WITH Intelligence:\n", with_i_summary.sort_values("metric").to_string(index=False))
print("\nWITHOUT Intelligence:\n", without_i_summary.sort_values("metric").to_string(index=False))
print("\nWITH - WITHOUT Intelligence comparison (means):\n", comparison.to_string(index=False))

WITH Intelligence:
               metric         condition      mean      var    median       p95       min       max  n_tests
        drift_dx_mps with_intelligence  0.060988 0.000296  0.056357  0.084921  0.039097  0.089390       10
        drift_dz_mps with_intelligence -0.072737 0.000561 -0.070793 -0.046821 -0.107082 -0.044238       10
 drift_heading_degps with_intelligence -0.003941 0.000006 -0.003431 -0.000898 -0.008218 -0.000198       10
   drift_pos_err_mps with_intelligence  0.008549 0.000217  0.009541  0.031671 -0.011004  0.034418       10
heading_err_deg_mean with_intelligence  0.128346 0.000527  0.125766  0.158076  0.093904  0.159711       10
 heading_err_deg_var with_intelligence  0.065106 0.000490  0.062241  0.096658  0.027684  0.115170       10
 pos_err_planar_mean with_intelligence  0.979107 0.014191  0.990641  1.125485  0.822111  1.161136       10
  pos_err_planar_var with_intelligence  0.412348 0.029644  0.456170  0.630377  0.142326  0.640235       10

WITHOUT Intellig

In [5]:
vehicle_converted_file = os.path.join(os.path.join(base_dir, "raw_virtual"), "Trial_1_Pedestrian (3rd Person)_Sent.txt")
vehicle_original_file = os.path.join(os.path.join(base_dir, "raw_virtual"), "Trial_1_pedestrian_to_vehicle.txt")

vconv_data = []
vorig_data = []

with open(vehicle_converted_file, "r") as f:
    for line in f:
        try:
            record = json.loads(line.strip())
            lonr, latr, alt = map(float, record["position"].split(", "))
            lat, lon = converter.convert(lonr, latr)
            vconv_data.append({
                "timestamp": float(record["timeStamp"]),
                "z": latr,
                "x": lonr,
                "heading": float(record["heading"])
            })
        except:
            continue

with open(vehicle_original_file, "r") as f:
    for line in f:
        try:
            record = json.loads(line.strip())
            lonr, latr, alt = map(float, record["position"].split(", "))
            lat, lon = converter.convert(lonr, latr)
            vorig_data.append({
                "timestamp": float(record["t_vir_obj_sent"]),
                "z": latr,
                "x": lonr,
                "heading": float(record["heading"])
            })
        except:
            continue

vconv_df_all = pd.DataFrame(vconv_data).dropna()
vorig_df_all = pd.DataFrame(vorig_data).dropna()

vconv_withi = []
vconv_withoi = []

vorig_withi = []
vorig_withoi = []

for condition in [withi, withouti]:
    condition_path = os.path.join(timestamp_dir, condition)
    for json_file in glob(os.path.join(condition_path, "*.json")):
        with open(json_file, "r") as f:
            data = json.load(f)
        entries = data.get("DataEntries", [])
        if not entries:
            continue
        test_start = float(entries[0]["Timestamp"])
        test_end = float(entries[-1]["Timestamp"])

        vconv_df = vconv_df_all[(vconv_df_all["timestamp"] >= test_start) & (vconv_df_all["timestamp"] <= test_end)].copy()
        vconv_df = vconv_df.sort_values("timestamp")
        vconv_df["elapsed_time"] = vconv_df["timestamp"] - test_start    

        vorig_df = vorig_df_all[(vorig_df_all["timestamp"] >= test_start) & (vorig_df_all["timestamp"] <= test_end)].copy()
        vorig_df = vorig_df.sort_values("timestamp")
        vorig_df["elapsed_time"] = vorig_df["timestamp"] - test_start  
        
        if(condition == "virtual_pedestrian_scenario_with_intelligence"):
            vconv_withi.append(vconv_df)
            vorig_withi.append(vorig_df)
        elif (condition == "virtual_pedestrian_scenario_without_intelligence"):
            vconv_withoi.append(vconv_df)
            vorig_withoi.append(vorig_df)


print(vconv_withi)
print("-----")
print(vconv_withoi)
print("-----")
print(vorig_withi)
print("-----")
print(vorig_withoi)

[           timestamp         z         x     heading  elapsed_time
201643  1.744960e+09 -63.71156  12.23688  267.924164         0.000
201644  1.744960e+09 -63.71156  12.23688  267.924164         0.016
201645  1.744960e+09 -63.71156  12.23688  267.924164         0.034
201646  1.744960e+09 -63.71156  12.23688  267.924164         0.050
201647  1.744960e+09 -63.71156  12.23688  267.924164         0.067
...              ...       ...       ...         ...           ...
203294  1.744960e+09 -65.04409 -15.59556  267.978271        27.510
203295  1.744960e+09 -65.04409 -15.59556  267.978271        27.530
203296  1.744960e+09 -65.04409 -15.59556  267.978271        27.549
203297  1.744960e+09 -65.04409 -15.59556  267.978271        27.568
203298  1.744960e+09 -65.04409 -15.59556  267.978271        27.578

[1656 rows x 5 columns],            timestamp         z         x     heading  elapsed_time
264821  1.744961e+09 -63.85301  11.50212  271.837830         0.001
264822  1.744961e+09 -63.85301  11.

In [6]:
import math
import numpy as np
import pandas as pd

PROJ_X_OFFSET = 356_422.983 + 1.63995
PROJ_Z_OFFSET =  4_028_673.294 + 0.3165

def _angle_wrap_deg(a):
    a = (a + 180.0) % 360.0 - 180.0
    return -180.0 if a == 180.0 else a

def _stats(s: pd.Series):
    a = pd.to_numeric(s, errors="coerce").dropna().to_numpy()
    if a.size == 0:
        return dict(mean=np.nan, var=np.nan, p95=np.nan, max=np.nan)
    return dict(
        mean=float(np.mean(a)),
        var=float(np.var(a, ddof=1)) if a.size > 1 else 0.0,
        p95=float(np.percentile(a, 95)),
        max=float(np.max(a)),
    )

def _linear_drift(time_s, resid):
    t = np.asarray(time_s, float)
    y = pd.to_numeric(resid, errors="coerce").to_numpy()
    m = np.isfinite(t) & np.isfinite(y)
    t, y = t[m], y[m]
    if t.size < 3:
        return np.nan
    x = t - t[0]
    if (x[-1] - x[0]) <= 1e-9:
        return np.nan
    slope, _ = np.polyfit(x, y, 1)
    return float(slope)

def real_to_unity(realPosX, realPosZ, realRotY_rad):
    playerRotY_deg = (360.0 - math.degrees(float(realRotY_rad))) % 360.0
    Xdisp = -((float(realPosZ) - PROJ_Z_OFFSET))
    Zdisp =  float(realPosX) - PROJ_X_OFFSET
    posX  = Xdisp
    posZ  = Zdisp
    return posX, posZ, playerRotY_deg

def _prep(df):
    d = df.copy().sort_values("timestamp").dropna(subset=["timestamp"]).reset_index(drop=True)
    for col in ["timestamp","x","z","heading"]:
        if col in d.columns:
            d[col] = pd.to_numeric(d[col], errors="coerce")
    return d.dropna(subset=["timestamp","x","z","heading"])

def calibration_error_one_test_exact_equal(df_orig, df_conv, dedupe="first"):
    o = _prep(df_orig)
    c = _prep(df_conv)

    if dedupe == "first":
        o = o.drop_duplicates("timestamp", keep="first")
        c = c.drop_duplicates("timestamp", keep="first")
    elif dedupe == "mean":
        o = o.groupby("timestamp", as_index=False).mean(numeric_only=True)
        c = c.groupby("timestamp", as_index=False).mean(numeric_only=True)
    else:
        raise ValueError("dedupe must be 'first' or 'mean'")

    aligned = o.merge(
        c[["timestamp","x","z","heading"]].rename(
            columns={"x":"x_conv","z":"z_conv","heading":"heading_conv"}
        ),
        on="timestamp", how="inner", suffixes=("_orig","_conv")
    ).sort_values("timestamp").reset_index(drop=True)

    if aligned.empty:
        return aligned, {
            "calib_err_planar_mean": np.nan, "calib_err_planar_var": np.nan,
            "calib_err_planar_p95": np.nan,  "calib_err_planar_max": np.nan,
            "calib_err_x_mean": np.nan,      "calib_err_x_var": np.nan,
            "calib_err_z_mean": np.nan,      "calib_err_z_var": np.nan,
            "calib_err_yaw_absdeg_mean": np.nan, "calib_err_yaw_absdeg_var": np.nan,
            "drift_err_x_mps": np.nan, "drift_err_z_mps": np.nan,
            "drift_err_planar_mps": np.nan, "drift_err_yaw_degps": np.nan,
            "n_matched": 0
        }

    x_pred, z_pred, yaw_pred = zip(*[
        real_to_unity(rx, rz, rhead) for rx, rz, rhead in
        zip(aligned["x"], aligned["z"], aligned["heading"])
    ])
    aligned["x_pred"] = np.array(x_pred, float)
    aligned["z_pred"] = np.array(z_pred, float)
    aligned["yaw_pred_deg"] = np.array(yaw_pred, float)

    aligned["err_x"] = aligned["x_pred"] - aligned["x_conv"]
    aligned["err_z"] = aligned["z_pred"] - aligned["z_conv"]
    aligned["err_planar"] = np.sqrt(aligned["err_x"]**2 + aligned["err_z"]**2)
    aligned["err_yaw_deg"] = (aligned["yaw_pred_deg"] - aligned["heading_conv"]).apply(_angle_wrap_deg)

    tsec = aligned["timestamp"].to_numpy(float)
    drift_x = _linear_drift(tsec, aligned["err_x"])
    drift_z = _linear_drift(tsec, aligned["err_z"])
    drift_e = _linear_drift(tsec, aligned["err_planar"])
    drift_h = _linear_drift(tsec, aligned["err_yaw_deg"])

    sX = _stats(aligned["err_x"]); sZ = _stats(aligned["err_z"])
    sE = _stats(aligned["err_planar"]); sH = _stats(aligned["err_yaw_deg"].abs())

    metrics = {
        "calib_err_planar_mean": sE["mean"],
        "calib_err_planar_var":  sE["var"],
        "calib_err_planar_p95":  sE["p95"],
        "calib_err_planar_max":  sE["max"],
        "calib_err_x_mean": sX["mean"],
        "calib_err_x_var":  sX["var"],
        "calib_err_z_mean": sZ["mean"],
        "calib_err_z_var":  sZ["var"],
        "calib_err_yaw_absdeg_mean": sH["mean"],
        "calib_err_yaw_absdeg_var":  sH["var"],
        "drift_err_x_mps": drift_x,
        "drift_err_z_mps": drift_z,
        "drift_err_planar_mps": drift_e,
        "drift_err_yaw_degps": drift_h,
        "n_matched": int(len(aligned)),
    }
    return aligned, metrics

def run_condition_exact_equal(vorig_list, vconv_list, label):
    rows = []
    for i, (df_o, df_c) in enumerate(zip(vorig_list, vconv_list), start=1):
        _, m = calibration_error_one_test_exact_equal(df_o, df_c, dedupe="first")
        m["test_id"] = i; m["condition"] = label
        rows.append(m)
    per_test = pd.DataFrame(rows)
    num = per_test.select_dtypes(include="number")
    summary = pd.DataFrame({
        "metric": num.columns,
        "condition": label,
        "mean": num.mean().values,
        "var":  num.var(ddof=1).values,
        "n_tests": len(vorig_list)
    })
    return per_test, summary

with_tests, with_summary = run_condition_exact_equal(vorig_withi, vconv_withi, "with_intelligence")
wo_tests,   wo_summary   = run_condition_exact_equal(vorig_withoi, vconv_withoi, "without_intelligence")

print("WITH Intelligence:\n", with_summary.sort_values("metric").to_string(index=False))
print("\nWITHOUT Intelligence:\n", wo_summary.sort_values("metric").to_string(index=False))

WITH Intelligence:
                    metric         condition       mean          var  n_tests
     calib_err_planar_max with_intelligence   0.235908 2.385835e-06       10
    calib_err_planar_mean with_intelligence   0.146907 7.347490e-04       10
     calib_err_planar_p95 with_intelligence   0.209854 3.058085e-04       10
     calib_err_planar_var with_intelligence   0.003158 1.195340e-06       10
         calib_err_x_mean with_intelligence   0.142339 7.595444e-04       10
          calib_err_x_var with_intelligence   0.003667 1.383100e-06       10
calib_err_yaw_absdeg_mean with_intelligence   0.000000 0.000000e+00       10
 calib_err_yaw_absdeg_var with_intelligence   0.000000 0.000000e+00       10
         calib_err_z_mean with_intelligence  -0.026907 9.298983e-06       10
          calib_err_z_var with_intelligence   0.000059 3.758862e-10       10
     drift_err_planar_mps with_intelligence  -0.002413 1.250564e-05       10
          drift_err_x_mps with_intelligence  -0.002591 1